In [3]:
from pathlib import Path

def make_sub_str(model_path, sweep_par, val, exp_dir):
    
    base_str = f'bsub -gpu "num=1" -q gpu_any -o {model_path}/{exp_dir}/logs/ocodsub.log -e {model_path}/{exp_dir}/logs/ecodesub.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py'    
    
    return base_str + ' ' + sweep_par + f'={val} run_name={exp_dir}/{sweep_par}:{val}' + '\n'

sweep1: 10000 iterations, bs 2, grad_clip 0.001, crop_sz 48, model ce
* group normalization helps (73.2 vs 71.5 with gce vs ce). cr > cr
* 5e-4 best lr
* crop_sz 48 slightly better than 40. 32 breaks down?

sweep2: 10000 iterations, bs 2, grad_clip 0.001, crop_sz 40, model gce
* Best performance without grad_clip
* Slight improvement with more channels (48>32>16)

sweep3: 10000 iterations, bs 2, grad_clip 0.001, crop_sz 48, model gce
* Microscope scale 10 -> 10000. Trying to match fish_sim dist.



In [4]:
model_path = Path('/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod')
exp_dir = 'ae1'

sweep_par = 'dataloader.bs'
sweep_vals = [1,2,4,6]

In [9]:
f'rm /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/{exp_dir}/logs/ecodesub.log'

'rm /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/ae1/logs/ecodesub.log'

In [8]:
Path.mkdir(model_path/exp_dir, exist_ok=True)
Path.mkdir(model_path/exp_dir/'logs', exist_ok=True)

with open('run_slurm.sh', 'w') as f:
    f.write(f'rm -r /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/fishcod/runs/{exp_dir} \n')
    f.write(f'rm /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/{exp_dir}/logs/ocodesub.log \n')
    f.write(f'rm /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/{exp_dir}/logs/ecodesub.log \n')
    for v in sweep_vals:  
        exec_str = make_sub_str(model_path, sweep_par, v, exp_dir)
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -q gpu_any -o /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/ae1/logs/ocodsub.log -e /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/ae1/logs/ecodesub.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py dataloader.bs=1 run_name=ae1/dataloader.bs:1

bsub -gpu "num=1" -q gpu_any -o /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/ae1/logs/ocodsub.log -e /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/ae1/logs/ecodesub.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py dataloader.bs=2 run_name=ae1/dataloader.bs:2

bsub -gpu "num=1" -q gpu_any -o /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/ae1/logs/ocodsub.l